In [1]:
import ast
import os
import sys

parent_dir = "/Volumes/work/phd/MoLFormer_N2024"
sys.path.append(parent_dir)
base_dir= '../../../../T5 EVO/'

In [2]:
import os
import numpy as np
import pandas as pd
base_path = '../../../../T5 EVO/alignment_olfaction_datasets/'
from sklearn.model_selection import cross_validate,train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
import scipy

In [3]:
def custom_ridge_regression(X, y, seed):
    linreg = RidgeCV()
    estimator = linreg.fit(X, y)
    return estimator

In [4]:
def metrics_per_descritor(X, y, linreg):
    predicted = linreg.predict(X)
    mseerrors = []
    correlations = []
    if len(y.shape) > 1:
        for i in range(y.shape[1]):
            mseerror = mean_squared_error(predicted[:, i], y[:, i])
            
            correlation = scipy.stats.pearsonr(predicted[:, i], y[:, i])
            
                
            mseerrors.append(mseerror)
            correlations.append(correlation)

    else:
        mseerror = mean_squared_error(predicted, y)
        correlation = scipy.stats.pearsonr(predicted, y)
        mseerrors.append(mseerror)
        correlations.append(correlation)
        
        

    return predicted, mseerrors, correlations

In [5]:
def select_features(input_file):
    ds_alva = pd.read_csv(input_file)


    chemical_features_r=["nCIR",
                     "ZM1", 
                     "GNar", 
                     "S1K", 
                     "piPC08",
                     "MATS1v",
                     "MATS7v",
                     "GATS1v", 
                     "Eig05_AEA(bo)", 
                     "SM02_AEA(bo)",
                     "SM03_AEA(dm)",
                     "SM10_AEA(dm)",
                     "SM13_AEA(dm)",
                      "SpMin3_Bh(v)",
                     "RDF035v",
                     "G1m",
                     "G1v",
                     "G1e",
                     "G3s",
                     "R8u+",
                     "nRCOSR"]

    nonStereoSMILE = list(map(lambda x: "nonStereoSMILES___" + x, chemical_features_r))
    # IsomericSMILES = list(map(lambda x: "IsomericSMILES___" + x, chemical_features_r))
    selected_features = nonStereoSMILE
    features= ['CID','nonStereoSMILES']+selected_features
    # print("cc1", ds_alva.columns.values.tolist())
    ds_alva= ds_alva.rename(columns={"cid":"CID"})
    # print("cc2", ds_alva.columns.values.tolist())
    ds_alva_selected = ds_alva[features]
    ds_alva_selected = ds_alva_selected.fillna(0)
    ds_alva_selected['embeddings'] = ds_alva_selected[selected_features].values.tolist()
    return ds_alva_selected


In [6]:
sagar_descriptors = ['0.1',
     '1.1',
     '2.1',
     '3.1',
     '4.1',
     '5.1',
     '6.1',
     '7.1',
     '8.1',
     '9.1',
     '10.1',
     '11.1',
     '12.1',
     '13.1',
     '14.1'
     ]


sagar_descriptors2 = [
     '0',
     '1',
     '2',
     '3',
     '4',
     '5',
     '6',
     '7',
     '8',
     '9',
     '10',
     '11',
     '12',
     '13',
     '14'
     ]

In [7]:
def prepare_dataset(ds,x_att,y_att):
    ds[y_att] = ds[y_att].apply(ast.literal_eval)
    ds[x_att] = ds[x_att].apply(ast.literal_eval)
    return ds

In [8]:
def grand_average(df, ds):
    if ds == "sagar":
        descriptors = sagar_descriptors
    elif ds == "sagar2":
        descriptors = sagar_descriptors2
    else:
        raise ValueError("Invalid dataset")

    df_groupbyCID = df.groupby('CID')[descriptors].mean().reset_index()

    df_groupbyCID['y'] = df_groupbyCID.loc[:, descriptors[0]:descriptors[-1]].values.tolist()
    df_embeddings = df.drop_duplicates(subset=['CID'])
    df_embeddings = df_embeddings[['CID', 'embeddings']]
    df_groupbyCID = pd.merge(df_groupbyCID, df_embeddings, on='CID', how='left')
    return df_groupbyCID


In [9]:
def train_and_eval(data,times,n_components=None):
    mserrorrs_corssvalidated = []
    correlations_corssvalidated = []
    predicteds = []
    y_tests = []
    runs = []
    CIDs = []
    
    
    
    X=np.asarray(data.embeddings.values.tolist())
    y=np.asarray(data.fmri_average.values.tolist())
    
    for i in range(times):
        X_train, X_test, y_train, y_test,CID_train, CID_test = train_test_split(X,y ,data.CID, test_size=0.2, random_state=seed+i) 
        # print(X_train.shape,X_test.shape)
        linreg = custom_ridge_regression(X_train, y_train, seed)
        
        
        predicted, mseerrors, correlations=metrics_per_descritor(X_test,y_test,linreg)
        mserrorrs_corssvalidated.append(mseerrors)
        correlations_corssvalidated.append(correlations)
        predicteds.extend(predicted)
        y_tests.extend(y_test)
        runs.extend([i]*len(y_test))
        CIDs.extend(CID_test)
        
        
    return CIDs,predicteds,y_tests,runs,mserrorrs_corssvalidated, correlations_corssvalidated

In [10]:
def post_process_results_df(mserrorrs_corssvalidated, correlations_corssvalidated):
    mserrorrs_corssvalidated_array = np.asarray(mserrorrs_corssvalidated)
    if len(mserrorrs_corssvalidated_array.shape) == 3:
        mserrorrs_corssvalidated_array = np.squeeze(mserrorrs_corssvalidated_array, -1)
        mserrorrs_corssvalidated_array = np.moveaxis(mserrorrs_corssvalidated_array, 0, 1)
    # print(mserrorrs_corssvalidated_array.shape,"shapeeee1")

    correlations_corssvalidated = np.asarray(correlations_corssvalidated)
    if len(correlations_corssvalidated.shape) == 4:
        correlations_corssvalidated = np.moveaxis(correlations_corssvalidated, 0, 1)
        # print("correlations_corssvalidateds",correlations_corssvalidated.shape)
        correlations_corssvalidated = np.squeeze(correlations_corssvalidated, 2)
    # print(correlations_corssvalidated.shape,"shapeeee2")
    statistics_correlations_corssvalidated_array = correlations_corssvalidated[:, :, 0]
    pvalues_correlations_corssvalidated_array = correlations_corssvalidated[:, :, 1]

    return mserrorrs_corssvalidated_array, statistics_correlations_corssvalidated_array, pvalues_correlations_corssvalidated_array



In [11]:
def pipeline(fmri_ev,voxel,model_name, input_file,input_file_alva=None,times=30,n_components=None):
    df_predictions,df_df_mse, df_df_cor = None,None,None
    
    # input_file_keller = base_path+'openpom/data/curated_datasets/embeddings/molformer/keller_molformer_embeddings_13_Apr17.csv'
    fmri_ev_voxel = fmri_ev[fmri_ev.voxel == voxel]
    df=pd.read_csv(input_file)
    df=prepare_dataset(df,'embeddings','y')
    df_groupbyCID=grand_average(df,'sagar')
    # df_groupbyCIDSubject=average_over_subject(df,'sagar')
    
    

    if input_file_alva is not None:
        
        df_alva = select_features(input_file_alva)
        df_alva = df_alva.drop_duplicates(subset=['CID'])
        del df_groupbyCID['embeddings']
        df_groupbyCID= pd.merge(df_alva,df_groupbyCID,on="CID")
    
        
    
    # X=np.asarray(df_groupbyCID.embeddings.values.tolist())
    #merge X, y on CID
    df = pd.merge(df_groupbyCID,fmri_ev_voxel,on='CID')
    
    
    
    CIDs, predicteds, y_tests,runs, mserrorrs_df_corssvalidated, correlations_df_corssvalidated=train_and_eval(df,times=times,n_components=n_components)
    # print(np.asarray(predicteds).shape,np.asarray(y_tests).shape, np.asarray(runs).shape, np.asarray(CIDs).shape)
    mserrorrs_corssvalidated_df,statistics_correlations_corssvalidated_df,pvalues_correlations_corssvalidated_df=post_process_results_df(mserrorrs_df_corssvalidated, correlations_df_corssvalidated)
    df_df_mse= pd.DataFrame(mserrorrs_corssvalidated_df)
    # df_df_mse = df_df_mse.T
    df_df_mse['model'] = model_name
    df_df_mse['voxel'] = voxel
    df_df_mse['type'] = 'mse'
    df_df_cor= pd.DataFrame(statistics_correlations_corssvalidated_df)
    df_df_cor['model'] = model_name
    df_df_cor['voxel'] = voxel
    df_df_cor['type'] = 'cor'
    
    #concatenate the mse and cor dataframes vertically
    df_metric = pd.concat([df_df_cor,df_df_mse])
    
    # print(np.asarray(predicteds).shape,np.asarray(y_tests).shape, np.asarray(runs).shape, np.asarray(CIDs).shape)

    # I want to make a dataframe with the predicted values, the true values and the run number for each prediction, (192, 22) (192, 22) (192,) should be converted to (196, 22+22+1), 
    df_predictions = pd.DataFrame(np.concatenate([np.asarray(CIDs).reshape(-1,1),np.asarray(predicteds),np.asarray(y_tests),np.asarray(runs).reshape(-1,1)],axis=1))
    df_predictions['model'] = model_name
    df_predictions['voxel'] = voxel
    #and add a prefix to the columns to indicate the predicted vs true values
    df_predictions.columns = ['CID']+[str(i)+'_predicted' for i in range(11)]+[str(i)+'_true' for i in range(11)]+['run']+['model']+['voxel']
    
    
    return df_predictions,df_metric

In [12]:
def compute_correlation(times,n_components,fmri_ev,input_file_molformer,input_file_pom,input_file_alva,input_file_molformerfinetuned):
    # metric = None,None,None,None
    # fmri_ev_voxel = fmri_ev[fmri_ev.voxel == voxel]
    # print("pom")
    metrics_molformer=[]
    metrics_molformerfinetuned = []
    metrics_pom=[]
    metrics_alva = [] 
    
    predictions_molformer=[]
    predictions_molformerfinetuned = []
    predictions_pom=[]
    predictions_alva = [] 

    for voxel in fmri_ev.voxel.unique():
        # print(voxel)
    # for voxel in [0,1,2]:
        
        df_predictions_pom, df_metric_pom = pipeline(fmri_ev,voxel,'pom',input_file_pom,times=times,n_components=n_components)
        metrics_pom.append(df_metric_pom)
        # predictions_pom.append(df_predictions_pom)
        

        df_predictions_alva,df_metric_alva = pipeline(fmri_ev,voxel, 'alva',input_file_pom,input_file_alva,times=times,n_components=None)
        metrics_alva.append(df_metric_alva)
        # predictions_alva.append(df_predictions_alva)
        
        # 
        
    
        # for i in [0,1,2,3,4,5,6,7,8,9,10,11,13]:
        for i in [13]:
        
            input_file_keller_molformer = input_file_molformer+str(i)+'_Apr17.csv'
            df_predictions_molformer,df_metric_molformer = pipeline(fmri_ev,voxel,'molformer',input_file_keller_molformer,times=times,n_components=n_components)
            
            df_predictions_molformer['layer'] = i
            df_metric_molformer['layer'] = i
            
    
            #append df_keller_cor_molformer and df_keller_mse_molformer 
        
            metrics_molformer.append(df_metric_molformer)            
            # predictions_molformer.append(df_predictions_molformer)


            # print("molformerfinetuned")
            # input_file_keller_molformerfinetuned = input_file_molformerfinetuned+str(i)+'_model_1_Apr17.csv'
            # 
            # 
            # df_predictions_molformerfinetuned,df_keller_mse_molformerfinetuned, df_keller_cor_molformerfinetuned = pipeline(fmri_ev,voxel, 'molformerfinetuned',input_file_keller_molformerfinetuned,times=times,n_components=n_components)
            # df_predictions_molformerfinetuned['layer'] = i
            # print("df_keller_molformer", df_keller_cor_molformerfinetuned.columns.values.tolist())
            # corrs_molformerfinetuned.append(df_keller_cor_molformerfinetuned)
            # mses_molformerfinetuned.append(df_keller_mse_molformerfinetuned)
            # df_predictions_molformerfinetuneds.append(df_predictions_molformerfinetuned)
    # 
    #    

            # df_predictions_pom['layer'] = 13
            # df_predictions_alva['layer'] = 13

    return metrics_molformer,metrics_molformerfinetuned,metrics_pom,metrics_alva,predictions_molformer,predictions_molformerfinetuned,predictions_pom,predictions_alva

In [13]:
# def post_process_tocsv(corrs,n_voxels,n_layers):
#     corrs[0]["layer"]=0
#     corrss = corrs[0]
#     # print(len(corrs),"len")
#     for i in range(1,len(corrs)):
#         # print("i", i )
#         corrs[i]["layer"] = i
#         corrss  = pd.concat([corrss, corrs[i]])
#     
#     return corrss

In [14]:
def save_data(ds,metrics_pom,metrics_alva,metrics_molformer,metrics_molfomerfinetuned):

    columns  = [str(i) for i in range(11)]
    #concatenate a list of dataframes vertically
    metrics_pom = pd.concat(metrics_pom)
    # metrics_pom.columns = columns+["model" , "voxel"]
    metrics_pom.to_csv(ds+'_pom.csv', index=False)  

    
    
    metrics_alva = pd.concat(metrics_alva)
    # metrics_alva.columns = columns+["model" , "voxel"]
    metrics_alva.to_csv(ds+'_alvanotnan.csv', index=False)

    metrics_molformer = pd.concat(metrics_molformer)
    # metrics_molformer.columns = columns+["model" , "voxel"]
    metrics_molformer.to_csv(ds+'_molfomer.csv', index=False)   


  

In [15]:
def safe_literal_eval(array_str):
# array_3d = np.array(list_3d)
#     cleaned_str = array_str.replace('\n', ',').replace(' ', ',').replace(',,', ',').replace(',,,', ',').replace(',,', '').replace('[,', '[').strip()
    cleaned_str = array_str.replace('\n', ',').replace('   ', ',').replace('  ', ',').replace(' ', ',').replace(',,,', ',').replace(',,', ',').replace('[,', '[')
#  
# # Step 3: Convert string back to a Python list (use ast.literal_eval for safety)
    try:
        list_3d = ast.literal_eval(cleaned_str)
        # print(np.asarray(list_3d).shape)
    except SyntaxError:
        print(f"SyntaxError: {array_str}")
        print(f"SyntaxError: {cleaned_str}")
# 
# # Step 4: Convert the list to a NumPy array
    array_3d = np.asarray(list_3d)
    return array_3d

def average(array_3d):
    return np.mean(array_3d,axis=0)

In [16]:
seed = 2024
input_file_sagar_pom = base_path+'curated_datasets/embeddings/pom/sagar_pom_embeddings_Apr17.csv'

input_file_sagar_alva = base_path+'curated_datasets/alva/sagar_molecules_alva_17Apr.csv'
input_file_sagar_mordred = base_path+'curated_datasets/mordred/sagar_molecules_mordred_17Apr.csv'
input_file_sagar_molformer = base_path+'curated_datasets/embeddings/molformer/sagar_molformer_embeddings_'
input_file_sagar_molformerfinetuned = base_path+'curated_datasets/embeddings/molformerfinetuned/sagar_molformerfinetuned_embeddings_'

In [17]:
def read_fmri_ev(roi,s):
    fmri_ev = pd.read_csv(f'{base_dir}fmri/csvs/fmrii_all_voxels_ev_{roi}_S{s}.csv')
    fmri_ev['ev'] = fmri_ev['ev'].apply(lambda x: eval(x)[0])
    fmri_ev.rename(columns={'condition':'CID'},inplace=True)
    fmri_ev['fmri'] = fmri_ev['fmri'].apply(safe_literal_eval)#convert fmri_ev['fmri'][0] to a 3d array
    fmri_ev['fmri_average'] = fmri_ev['fmri'].apply(average)
    return fmri_ev

In [18]:
for i in [1,2,3]:
    for roi in [ 'APC', 'Amygdala', 'PPC']:
        print(i,roi)
        fmri_ev = read_fmri_ev(roi,i )
        times =30
        n_components = 20
        metrics_molformer,metrics_molformerfinetuned,metrics_pom,metrics_alva,predictions_molformer,predictions_molformerfinetuned,predictions_pom,predictions_alva=compute_correlation(times , n_components,fmri_ev[['fmri_average','CID','voxel']],input_file_sagar_molformer, input_file_sagar_pom,input_file_sagar_alva,input_file_sagar_molformerfinetuned)
        save_data(f'{base_dir}fmri/csvs/fmri_sagar_metrics_{roi}_S{i}',metrics_pom,metrics_alva,metrics_molformer,metrics_molformerfinetuned)   

2 APC
2 Amygdala
2 PPC


/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInpu

3 APC
3 Amygdala
3 PPC
